# 이미지 생성/편집 통합 테스트 (S3 업로드 포함)

실제 S3 업로드를 포함한 전체 워크플로우 테스트입니다.

**테스트 흐름**:
1. 새 캐릭터 이미지 생성 (Nova Canvas) → S3 업로드
2. S3에서 이미지 다운로드 → Gemini로 편집 → S3 업로드
3. 시간 흐름 반영 (10년 후) → S3 업로드

**테스트 대상 서비스:**
- `PromptService`: Claude를 이용한 프롬프트 최적화
- `BedrockService`: Nova Canvas 이미지 생성
- `GeminiService`: Gemini (Nano Banana) 이미지 편집
- `S3Service`: AWS S3 이미지 업로드/다운로드

> ⚠️ **주의**: 이 테스트는 실제 S3에 이미지를 업로드합니다. AWS 비용이 발생할 수 있습니다.

## 1. 환경 설정

In [1]:
import os
import sys
from pathlib import Path

# 프로젝트 루트를 sys.path에 추가
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

print(f"프로젝트 루트: {PROJECT_ROOT}")

프로젝트 루트: c:\jungle\weapon\sto-link-image-backend


In [2]:
# .env 파일 로드
from dotenv import load_dotenv

env_path = PROJECT_ROOT / '.env'
load_dotenv(env_path)

# 환경 변수 확인
print("환경 변수 확인:")
print(f"  AWS_ACCESS_KEY_ID: {'✅ 설정됨' if os.getenv('AWS_ACCESS_KEY_ID') else '❌ 미설정'}")
print(f"  AWS_SECRET_ACCESS_KEY: {'✅ 설정됨' if os.getenv('AWS_SECRET_ACCESS_KEY') else '❌ 미설정'}")
print(f"  AWS_REGION: {os.getenv('AWS_REGION', '❌ 미설정')}")
print(f"  AWS_S3_BUCKET_NAME: {os.getenv('AWS_S3_BUCKET_NAME', '❌ 미설정')}")
print(f"  AWS_S3_REGION: {os.getenv('AWS_S3_REGION', '❌ 미설정')}")
print(f"  GEMINI_API_KEY: {'✅ 설정됨' if os.getenv('GEMINI_API_KEY') else '❌ 미설정'}")

환경 변수 확인:
  AWS_ACCESS_KEY_ID: ✅ 설정됨
  AWS_SECRET_ACCESS_KEY: ✅ 설정됨
  AWS_REGION: us-east-1
  AWS_S3_BUCKET_NAME: novel-project-images
  AWS_S3_REGION: us-east-1
  GEMINI_API_KEY: ✅ 설정됨


## 2. 서비스 임포트 및 초기화

In [3]:
from app.services.prompt_service import get_prompt_service
from app.services.bedrock_service import get_bedrock_service
from app.services.gemini_service import get_gemini_service
from app.services.s3_service import get_s3_service

# 서비스 초기화
prompt_service = get_prompt_service()
bedrock_service = get_bedrock_service()
gemini_service = get_gemini_service()
s3_service = get_s3_service()

print("✅ 모든 서비스 초기화 완료")
print(f"  - PromptService: Claude {bedrock_service.claude_model_id}")
print(f"  - BedrockService: Nova Canvas {bedrock_service.nova_canvas_model_id}")
print(f"  - GeminiService: {gemini_service.model_id}")
print(f"  - S3Service: Bucket={s3_service.bucket_name}, Region={s3_service.region}")

✅ 모든 서비스 초기화 완료
  - PromptService: Claude us.anthropic.claude-3-5-haiku-20241022-v1:0
  - BedrockService: Nova Canvas amazon.nova-canvas-v1:0
  - GeminiService: gemini-2.5-flash-image
  - S3Service: Bucket=novel-project-images, Region=us-east-1


## 3. 유틸리티 함수

In [4]:
from IPython.display import Image, display, HTML
import base64

def show_image_from_url(url: str, title: str = ""):
    """URL에서 이미지를 다운로드하여 표시"""
    if title:
        print(f"\n=== {title} ===")
    print(f"URL: {url}")
    
    # S3에서 이미지 다운로드
    image_bytes = s3_service.download_image(url)
    
    # Base64로 인코딩하여 표시
    img_base64 = base64.b64encode(image_bytes).decode()
    display(HTML(f'<img src="data:image/png;base64,{img_base64}" style="max-width: 400px;">'))

def show_image_from_bytes(image_bytes: bytes, title: str = ""):
    """바이트 데이터에서 이미지 표시"""
    if title:
        print(f"\n=== {title} ===")
    img_base64 = base64.b64encode(image_bytes).decode()
    display(HTML(f'<img src="data:image/png;base64,{img_base64}" style="max-width: 400px;">'))

---
## 4. Step 1: 캐릭터 베이스 이미지 생성 → S3 업로드

In [5]:
print("="*60)
print("Step 1: 캐릭터 베이스 이미지 생성")
print("="*60)

# 1.1 프롬프트 생성
character_description = "검은색 정장을 입은 카리스마 있는 20대 한국인 남성"
print(f"\n📝 캐릭터 설명: {character_description}")

print("\n⏳ Claude로 프롬프트 생성 중...")
enhanced_prompt = prompt_service.create_character_prompt(character_description)
print(f"✅ 생성된 프롬프트: {enhanced_prompt[:100]}...")

Step 1: 캐릭터 베이스 이미지 생성

📝 캐릭터 설명: 검은색 정장을 입은 카리스마 있는 20대 한국인 남성

⏳ Claude로 프롬프트 생성 중...
✅ 생성된 프롬프트: A high-resolution ID-style portrait of a charismatic Korean man in his 20s, wearing a sleek black su...


In [6]:
# 1.2 Nova Canvas로 이미지 생성
print("\n🎨 Nova Canvas 이미지 생성 중... (30초~1분 소요)")

base_image_bytes = bedrock_service.invoke_nova_canvas(
    prompt=enhanced_prompt,
    negative_prompt="blurry, distorted, low quality, deformed face"
)

print(f"✅ 이미지 생성 완료: {len(base_image_bytes):,} bytes")


🎨 Nova Canvas 이미지 생성 중... (30초~1분 소요)
✅ 이미지 생성 완료: 1,569,206 bytes


In [7]:
# 1.3 S3에 업로드
print("\n☁️ S3에 업로드 중...")

base_image_url = s3_service.upload_image(
    image_bytes=base_image_bytes,
    prefix="character"
)

print(f"✅ S3 업로드 완료!")
print(f"   URL: {base_image_url}")

# 이미지 표시
show_image_from_bytes(base_image_bytes, "베이스 캐릭터 이미지")


☁️ S3에 업로드 중...


Failed to upload image to S3: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied


ClientError: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied

---
## 5. Step 2: 특징 수정 (Gemini) → S3 업로드

In [ ]:
print("="*60)
print("Step 2: 특징 수정 (장발로 변경)")
print("="*60)

# 2.1 편집 프롬프트 생성
edit_request = "기존 인물과 얼굴은 똑같이 유지하되, 머리 길이를 어깨까지 오는 장발로 바꿔줘."
print(f"\n📝 편집 요청: {edit_request}")

print("\n⏳ Claude로 편집 프롬프트 생성 중...")
edit_prompt = prompt_service.create_edit_prompt(edit_request)
print(f"✅ 생성된 프롬프트: {edit_prompt}")

In [ ]:
# 2.2 S3에서 원본 이미지 다운로드
print("\n📥 S3에서 원본 이미지 다운로드 중...")
source_image_bytes = s3_service.download_image(base_image_url)
print(f"✅ 다운로드 완료: {len(source_image_bytes):,} bytes")

In [ ]:
# 2.3 Gemini로 이미지 편집
print("\n✏️ Gemini 이미지 편집 중... (30초~1분 소요)")

edited_image_bytes = gemini_service.edit_image(
    image_bytes=source_image_bytes,
    edit_prompt=edit_prompt
)

print(f"✅ 이미지 편집 완료: {len(edited_image_bytes):,} bytes")

In [ ]:
# 2.4 S3에 업로드
print("\n☁️ S3에 업로드 중...")

edited_image_url = s3_service.upload_image(
    image_bytes=edited_image_bytes,
    prefix="edited_feature"
)

print(f"✅ S3 업로드 완료!")
print(f"   URL: {edited_image_url}")

# 이미지 표시
show_image_from_bytes(edited_image_bytes, "특징 수정 이미지 (장발)")

---
## 6. Step 3: 시간 흐름 반영 (10년 후) → S3 업로드

In [ ]:
print("="*60)
print("Step 3: 시간 흐름 반영 (10년 후)")
print("="*60)

# 3.1 편집 프롬프트 생성
age_edit_request = "이 인물이 10년 후 모습을 보여줘. 머리에 흰머리가 섞이고 자연스럽게 나이 든 모습으로."
print(f"\n📝 편집 요청: {age_edit_request}")

print("\n⏳ Claude로 편집 프롬프트 생성 중...")
age_edit_prompt = prompt_service.create_edit_prompt(age_edit_request)
print(f"✅ 생성된 프롬프트: {age_edit_prompt}")

In [ ]:
# 3.2 S3에서 원본 이미지 다운로드 (베이스 이미지 기준)
print("\n📥 S3에서 원본 이미지 다운로드 중...")
source_bytes_for_aging = s3_service.download_image(base_image_url)
print(f"✅ 다운로드 완료: {len(source_bytes_for_aging):,} bytes")

In [ ]:
# 3.3 Gemini로 이미지 편집
print("\n✏️ Gemini 이미지 편집 중... (30초~1분 소요)")

aged_image_bytes = gemini_service.edit_image(
    image_bytes=source_bytes_for_aging,
    edit_prompt=age_edit_prompt
)

print(f"✅ 이미지 편집 완료: {len(aged_image_bytes):,} bytes")

In [ ]:
# 3.4 S3에 업로드
print("\n☁️ S3에 업로드 중...")

aged_image_url = s3_service.upload_image(
    image_bytes=aged_image_bytes,
    prefix="edited_aged"
)

print(f"✅ S3 업로드 완료!")
print(f"   URL: {aged_image_url}")

# 이미지 표시
show_image_from_bytes(aged_image_bytes, "10년 후 이미지")

---
## 7. 결과 요약

In [ ]:
print("=" * 60)
print("🎉 전체 테스트 완료!")
print("=" * 60)

print("\n📁 S3 Bucket:", s3_service.bucket_name)
print("\n✅ 업로드된 이미지 URL:")

try:
    print(f"  1. 베이스 이미지: {base_image_url}")
except NameError:
    print("  1. 베이스 이미지: (미생성)")

try:
    print(f"  2. 특징 수정 이미지: {edited_image_url}")
except NameError:
    print("  2. 특징 수정 이미지: (미생성)")

try:
    print(f"  3. 10년 후 이미지: {aged_image_url}")
except NameError:
    print("  3. 10년 후 이미지: (미생성)")

## 8. 이미지 비교

In [ ]:
try:
    # 각 이미지를 base64로 인코딩
    base_b64 = base64.b64encode(base_image_bytes).decode()
    edited_b64 = base64.b64encode(edited_image_bytes).decode()
    aged_b64 = base64.b64encode(aged_image_bytes).decode()
    
    html = f"""
    <div style="display: flex; justify-content: space-around; gap: 10px; flex-wrap: wrap;">
        <div style="text-align: center;">
            <img src="data:image/png;base64,{base_b64}" style="max-width: 280px;">
            <p><b>1. 베이스 이미지</b></p>
            <p style="font-size: 10px; color: gray;">Nova Canvas</p>
        </div>
        <div style="text-align: center;">
            <img src="data:image/png;base64,{edited_b64}" style="max-width: 280px;">
            <p><b>2. 특징 수정 (장발)</b></p>
            <p style="font-size: 10px; color: gray;">Gemini Edit</p>
        </div>
        <div style="text-align: center;">
            <img src="data:image/png;base64,{aged_b64}" style="max-width: 280px;">
            <p><b>3. 10년 후</b></p>
            <p style="font-size: 10px; color: gray;">Gemini Edit</p>
        </div>
    </div>
    """
    display(HTML(html))
except NameError:
    print("⚠️ 이미지 비교를 위해서는 먼저 위의 테스트 셀들을 실행해주세요.")

---
## 9. LangGraph 통합 테스트 (선택사항)

실제 `ImageService`의 LangGraph 워크플로우를 테스트합니다.

In [ ]:
from app.graph.image_graph import get_image_graph

print("🔧 LangGraph 이미지 그래프 초기화...")
image_graph = get_image_graph()
print("✅ 그래프 초기화 완료")

In [ ]:
# LangGraph로 새 캐릭터 생성
print("="*60)
print("LangGraph 테스트: 캐릭터 이미지 생성")
print("="*60)

create_state = {
    "action": "create",
    "message": "파란색 후드티를 입은 밝은 표정의 20대 한국인 여성",
    "source_image_url": None,
    "enhanced_prompt": None,
    "edit_prompt": None,
    "generated_image_bytes": None,
    "result_image_url": None,
    "error": None,
    "job_id": "test-job-langgraph",
    "character_id": "test-char-langgraph",
}

print(f"입력 메시지: {create_state['message']}")
print("\n⏳ LangGraph 실행 중...")

create_result = image_graph.invoke(create_state)

if create_result.get("error"):
    print(f"\n❌ 오류 발생: {create_result['error']}")
else:
    print(f"\n✅ 성공!")
    print(f"   S3 URL: {create_result['result_image_url']}")
    
    # 이미지 표시
    if create_result.get('generated_image_bytes'):
        show_image_from_bytes(create_result['generated_image_bytes'], "LangGraph 생성 이미지")

In [ ]:
# LangGraph로 이미지 편집
if create_result.get("result_image_url") and not create_result.get("error"):
    print("="*60)
    print("LangGraph 테스트: 이미지 편집")
    print("="*60)
    
    edit_state = {
        "action": "edit",
        "message": "이 인물의 머리색을 보라색으로 바꿔줘",
        "source_image_url": create_result['result_image_url'],
        "enhanced_prompt": None,
        "edit_prompt": None,
        "generated_image_bytes": None,
        "result_image_url": None,
        "error": None,
        "job_id": "test-job-langgraph-edit",
        "character_id": "test-char-langgraph",
    }
    
    print(f"편집 요청: {edit_state['message']}")
    print(f"원본 이미지: {edit_state['source_image_url']}")
    print("\n⏳ LangGraph 실행 중...")
    
    edit_result = image_graph.invoke(edit_state)
    
    if edit_result.get("error"):
        print(f"\n❌ 오류 발생: {edit_result['error']}")
    else:
        print(f"\n✅ 성공!")
        print(f"   S3 URL: {edit_result['result_image_url']}")
        
        if edit_result.get('generated_image_bytes'):
            show_image_from_bytes(edit_result['generated_image_bytes'], "LangGraph 편집 이미지")
else:
    print("⚠️ 먼저 위 셀에서 이미지를 생성해주세요.")